In [ ]:
!git clone https://github.com/SKTBrain/KoBERT.git

Cloning into 'KoBERT'...
remote: Enumerating objects: 155, done.
remote: Total 155 (delta 0), reused 0 (delta 0), pack-reused 155
Receiving objects: 100% (155/155), 179.81 KiB | 8.99 MiB/s, done.
Resolving deltas: 100% (82/82), done.


In [ ]:
cd KoBERT

/content/KoBERT/KoBERT


In [ ]:
!pip install -r requirements.txt
!pip install .

Processing /content/KoBERT/KoBERT
  Created wheel for kobert: filename=kobert-0.1.2-cp36-none-any.whl size=12734 sha256=5b98db816cdfad46b471bf46daa9659b2cc7934d2bae592adc28603d42878c4c
  Stored in directory: /tmp/pip-ephem-wheel-cache-x7l3lhms/wheels/cd/9c/c8/68129cdbd7e1c11968964aa15522594a68516572354106cfa4
Successfully built kobert
  Found existing installation: kobert 0.1.2
    Uninstalling kobert-0.1.2:
      Successfully uninstalled kobert-0.1.2


In [ ]:
!pip install gluonnlp pandas tqdm
!pip install sentencepiece

In [ ]:
import os
os.chdir('/content/리뽀/')           # 저희 팀 이름인 '리뽀' 폴더를 content 폴더에 넣어 둔 채로 실행한다고 가정하여 작성하였습니다.

In [ ]:
# 데이터 불러오기

import pandas as pd

test_data = pd.read_csv('1. Data/news_test.csv')
submission_data = pd.read_csv('1. Data/sample_submission.csv')

In [ ]:
# 시간 측정 시작

import time
start = time.time()  # 시작 시간 저장

In [ ]:
# Library 불러오기

from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences  
import pickle
from tensorflow.keras.models import load_model

In [ ]:
# pos_tagger, tokenizer, pretrained_embedding, model 불러오기

tok_path = get_tokenizer()
sp = SentencepieceTokenizer(tok_path)                              # pos_tagger

with open('3. Tokenizer/tokenizer.pickle', 'rb') as handle:        # tokenizer
  loaded_tokenizer = pickle.load(handle)

np.load('4. Pre_trained embedding/embedding_matrix.npy')           # pre_trained embedding

loaded_model = load_model('5. Model/kobert_gensim_model.h5')       # model

[██████████████████████████████████████████████████]


In [ ]:
# 형태소 분석 + 전처리

x_test_data = test_data['content']

for i in range(0, len(x_test_data)):
  x_test_data[i] = sp(x_test_data[i])
  
tokenizer = loaded_tokenizer
x_Test_sequences = tokenizer.texts_to_sequences(x_test_data)

vocabulary = tokenizer.word_index
vocab_size = len(vocabulary) + 1

Test_data = x_Test_sequences

max_len = 25
X_test = pad_sequences(Test_data, maxlen = max_len)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# 예측

pred_test = loaded_model.predict(X_test)
submission_data.loc[:, 'info'] = np.where(pred_test > 0.5, 1,0).reshape(-1)

In [ ]:
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

time : 110.38749051094055


In [ ]:
# 제출

submission_data.loc[:, ["id","info"]].to_csv("submission_data(KoBERT)(gensim)1228(noval)ep1-newpara.csv", index = False)